In [1]:
import os
from pymongo import MongoClient
import pandas as pd
import json
import requests
from dotenv import load_dotenv
import re
load_dotenv()

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


True

In [2]:
startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'
def initDB():
    os.popen(startmongo)
os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db,coll = (list(connectCollection('companies','companies')))
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
value =  rest[0].get('total_money_raised')

In [5]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
clean_db = []
for i in range(len(rest)):
    value = rest[i].get('total_money_raised') 
    valueM = re.findall(r'\d+[?\.\d]?\d',value)
    if valueM == []:
        valueM = [0]
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': {
            'value':float(valueM[0]),
            'qty':value[-1]
        }
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)

In [6]:
with open('Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)

In [7]:
#command = 'mongoimport --db companies --collection companies_clean --jsonArray --file database_clean.json'
#os.popen(command)

'''def requestfoursquare(query,lon,lat,radius):
    '''
    #lon => float
    #lat => float
    #radius => meters
    '''
    url = 'https://api.foursquare.com/v2/venues/explore'
    #client = os.getenv("CLIENT_ID")
    #passw = os.getenv("CLIENT_SECRET")
    params = dict(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("CLIENT_SECRET"),
        v='20180335',
        ll='{},{}'.format(lat,lon),
        query=query,
        limit=200,
        radius = radius
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    return data'''
    
'''def getlocation(data):
    locationslist = []
    for i in range(len(data['response']['groups'][0]['items'])):
        latitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lat'))
        longitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lng'))
        dictio = {
            'name':(data['response']['groups'][0]['items'][i]['venue']).get('name'),
            'loc':{'type':'Point','coordinates':[longitude,latitude]}            
            }
        locationslist.append(dictio)
    return locationslist
'''

'''#'name: Starbucks'
for i in range(len(clean_db)):
    for ind in range(1,(len(clean_db[i])-2)):
        print('enterprise{}, location{}'.format(i,ind))
        lat = clean_db[i]['Location-{}'.format(ind)]['coordinates'][0]
        lon = clean_db[i]['Location-{}'.format(ind)]['coordinates'][1]
        data = requestfoursquare('type: Starbucks',lat,lon,1000)
        display(data)
        geo = getlocation(data)
        print(len(geo))
        break'''


In [8]:
load_dotenv()
db,coll_clean = (list(connectCollection('companies','companies_clean2')))

In [9]:
def gsearch(lat,lon,radius,query):
    URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params={
        'name':str(query),
        'location':'{},{}'.format(lat,lon),
        'radius':str(radius),
        'key':os.getenv('GOOGLE_KEY'),
        }
    res=requests.get(URL, params=params).json()
    return res

In [10]:
def geolocation(data):
    list_places = []
    for i in range(len(data['results'])):
        diction={}
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
        list_places.append(diction)
    return list_places

In [11]:
def getnearbyplaces(coll,radius,query,init = 1132):
    coll_list = (list(coll.find({})))
    try:
        init = init
        for i in range(init,len(coll_list)):
            print(f'enterprise{i}')
            collitem =  coll_list[i][f'Location-1']
            lat = collitem['coordinates'][1]
            lon = collitem['coordinates'][0]
            data = gsearch(lat,lon,radius,query)
            geoloc = geolocation(data)
            locations = geoloc
            places = len(geoloc)
            value = {"$set":{'Location-1':{'type':'Point','coordinates':[float(lat), float(lon)],query:{'locations':locations,'Nº':places}}}}
            coll.update_one(coll_list[i],value)
    except KeyError as e:
        init = i+1
        print(f'Error,{e}')
        return getnearbyplaces(coll_clean,500,'starbucks',init)

In [ ]:
getnearbyplaces(coll_clean,500,'starbucks',7843)

enterprise7843
enterprise7844
Error,'Location-1'
enterprise7845
enterprise7846
enterprise7847
enterprise7848
enterprise7849
enterprise7850
Error,'Location-1'
enterprise7851
enterprise7852
enterprise7853
Error,'Location-1'
enterprise7854
enterprise7855
Error,'Location-1'
enterprise7856
Error,'Location-1'
enterprise7857
Error,'Location-1'
enterprise7858
Error,'Location-1'
enterprise7859
enterprise7860
Error,'Location-1'
enterprise7861
enterprise7862
enterprise7863
enterprise7864
enterprise7865
enterprise7866
enterprise7867
Error,'Location-1'
enterprise7868
Error,'Location-1'
enterprise7869
enterprise7870
Error,'Location-1'
enterprise7871
enterprise7872
enterprise7873
enterprise7874
enterprise7875
enterprise7876
Error,'Location-1'
enterprise7877
enterprise7878
Error,'Location-1'
enterprise7879
enterprise7880
Error,'Location-1'
enterprise7881
enterprise7882
enterprise7883
enterprise7884
enterprise7885
enterprise7886
enterprise7887
enterprise7888
enterprise7889
enterprise7890
Error,'Locatio

enterprise8190
Error,'Location-1'
enterprise8191
enterprise8192
enterprise8193
enterprise8194
enterprise8195
Error,'Location-1'
enterprise8196
Error,'Location-1'
enterprise8197
enterprise8198
Error,'Location-1'
enterprise8199
Error,'Location-1'
enterprise8200
enterprise8201
Error,'Location-1'
enterprise8202
enterprise8203
enterprise8204
enterprise8205
enterprise8206
enterprise8207
enterprise8208
enterprise8209
enterprise8210
Error,'Location-1'
enterprise8211
Error,'Location-1'
enterprise8212
enterprise8213
enterprise8214
enterprise8215
Error,'Location-1'
enterprise8216
Error,'Location-1'
enterprise8217
Error,'Location-1'
enterprise8218
Error,'Location-1'
enterprise8219
Error,'Location-1'
enterprise8220
Error,'Location-1'
enterprise8221
Error,'Location-1'
enterprise8222
Error,'Location-1'
enterprise8223
Error,'Location-1'
enterprise8224
Error,'Location-1'
enterprise8225
Error,'Location-1'
enterprise8226
Error,'Location-1'
enterprise8227
Error,'Location-1'
enterprise8228
Error,'Location-

In [30]:
coll_list = (list(coll_clean.find({})))
display(coll_list[7])

{'_id': ObjectId('5dd50e1cbae979d9c80f57f0'),
 'name': 'Facebook',
 'category': 'social',
 'value': {'value': 2.4, 'qty': 'B'},
 'Location-1': {'type': 'Point', 'coordinates': [-122.151801, 37.41605]},
 'Location-2': {'type': 'Point', 'coordinates': [-6.267494, 53.344104]},
 'Location-3': {'type': 'Point', 'coordinates': [-73.9792469, 40.7557162]}}

In [32]:
transport = pd.read_csv("Database/airports-extended.csv")

In [33]:
transport = transport[transport.Type!= 'unknown']
transport

,Name,Lat,Lng,Type
0,Goroka Airport,-6.081690,145.391998,airport
1,Madang Airport,-5.207080,145.789001,airport
2,Mount Hagen Kagamuga Airport,-5.826790,144.296005,airport
3,Nadzab Airport,-6.569803,146.725977,airport
4,Port Moresby Jacksons International Airport,-9.443380,147.220001,airport
...,...,...,...,...
10651,Duisburg Hauptbahnhof,51.429402,6.774830,station
10657,Bamburi Airport,-3.979444,39.731388,airport
10658,Pioltello-Limito Railway Station,45.486111,9.329444,station
10659,Verdello-Dalmine Railway Station,45.605556,9.619722,station


In [37]:
geocode = []
for i in range(len(transport["Lat"])):
    lat = transport["Lat"].iloc[i]
    lng = transport["Lng"].iloc[i]
    diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
    geocode.append(diction)

In [38]:
transport['geocode'] = geocode

In [ ]:
from geopy.distance import geodesic 
def getdistance(location1,location2):
    return geodesic(location1, location2).km

In [ ]:
transport['geocode']